In [1]:
# 加载数据
import torch
from joblib import dump, load
import torch.utils.data as Data
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# 参数与配置
torch.manual_seed(100)  # 设置随机种子，以使实验结果具有可重复性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载数据集
def dataloader(batch_size, workers=2):
    # 训练集
    train_set = load('../dataresult/train_set')
    train_label = load('../dataresult/train_label')
    # 测试集
    test_set = load('../dataresult/test_set')
    test_label = load('../dataresult/test_label')

    # 加载数据
    train_loader = Data.DataLoader(dataset=Data.TensorDataset(train_set, train_label),
                                   batch_size=batch_size, num_workers=workers, drop_last=True)
    test_loader = Data.DataLoader(dataset=Data.TensorDataset(test_set, test_label),
                                  batch_size=batch_size, num_workers=workers, drop_last=True)
    return train_loader, test_loader

batch_size = 16
# 加载数据
train_loader, test_loader = dataloader(batch_size)

print(len(train_loader))
print(len(test_loader))

16
9


In [2]:
# 定义裁剪模块
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

#  定义 TCN 卷积+残差 模块
from torch.nn.utils import parametrizations
import torch
import torch.nn as nn
from bayesian_torch.layers import Conv1dReparameterization, LinearReparameterization

class BayesianTemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding, dropout=0.2):
        super().__init__()
        self.conv1 = Conv1dReparameterization(
            in_channels, out_channels, kernel_size,
            stride=stride, padding=padding, dilation=dilation,
            prior_mean=0, prior_variance=0.1, posterior_mu_init=0, posterior_rho_init=-3
        )
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = Conv1dReparameterization(
            out_channels, out_channels, kernel_size,
            stride=stride, padding=padding, dilation=dilation,
            prior_mean=0, prior_variance=0.1, posterior_mu_init=0, posterior_rho_init=-3
        )
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.final_relu = nn.ReLU()

    def forward(self, x):
        kl = 0.0
        out, kl1 = self.conv1(x)
        out = self.chomp1(out)
        kl += kl1
        out = self.relu1(out)
        out = self.dropout1(out)
        out, kl2 = self.conv2(out)
        out = self.chomp2(out)
        kl += kl2
        out = self.relu2(out)
        out = self.dropout2(out)
        res = x if self.downsample is None else self.downsample(x)
        return self.final_relu(out + res), kl

In [3]:
class BayesianTCN(nn.Module):
    def __init__(self, input_dim, num_channels, kernel_size=2, dropout=0.2, output_dim=1):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        self.kl_modules = []
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = input_dim if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            block = BayesianTemporalBlock(
                in_channels, out_channels, kernel_size,
                stride=1, dilation=dilation_size,
                padding=(kernel_size-1)*dilation_size,
                dropout=dropout
            )
            layers.append(block)
        self.network = nn.ModuleList(layers)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        # self.fc = LinearReparameterization(
        #     in_features=num_channels[-1], out_features=output_dim,
        #     prior_mean=0, prior_variance=0.1, posterior_mu_init=0, posterior_rho_init=-3
        # )
        self.mu = LinearReparameterization(
            in_features=num_channels[-1], out_features=output_dim,
            prior_mean=0, prior_variance=0.1, posterior_mu_init=0, posterior_rho_init=-3
        )
        # sigma是对数方差
        self.sigma = LinearReparameterization(
            in_features=num_channels[-1], out_features=output_dim,
            prior_mean=0, prior_variance=0.1, posterior_mu_init=0, posterior_rho_init=-3
        )


    def forward(self, x):
        # x: [batch, seq_len, input_dim] -> [batch, input_dim, seq_len]
        x = x.permute(0, 2, 1)
        kl = 0.0
        for block in self.network:
            x, kl_block = block(x)
            kl += kl_block
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        # out, kl_fc = self.fc(x)
        mu, kl_mu = self.mu(x)
        sigma, kl_sigma = self.sigma(x)

        kl += (kl_mu+kl_sigma)
        return mu, sigma, kl

    def kl_loss(self):
        kl = 0.0
        for m in self.children():  # 只遍历直接子模块
            if hasattr(m, "kl_loss"):
                kl += m.kl_loss()
        return kl

In [4]:
import sys
sys.path.append('..')
from loss_function import compute_au_nll
# 输入数据维度为[batch_size, sequence_length, input_size]
# 输出维度为[batch_size, output_dim]
input_dim = 14         # 输入特征维度
output_dim = 1         # 输出特征维度
num_channels = [32, 64]  # 每个TemporalBlock的输出通道数
kernel_size = 3        # 卷积核大小
dropout = 0.5          # Dropout概率
kl_weight = 0.001
model = BayesianTCN(
    input_dim=input_dim,
    num_channels=[32, 64],  # 这里必须是列表
    kernel_size=kernel_size,
    dropout=dropout,
    output_dim=output_dim
)

# 定义损失函数和优化函数 
# loss_function = nn.MSELoss()  # loss
loss_function = compute_au_nll
learn_rate = 0.0003
optimizer = torch.optim.Adam(model.parameters(), learn_rate)  # 优化器

# 看下这个网络结构总共有多少个参数
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')

count_parameters(model)

  1344
  1344
    32
    32
  3072
  3072
    32
    32
   448
    32
  6144
  6144
    64
    64
 12288
 12288
    64
    64
  2048
    64
    64
    64
     1
     1
    64
    64
     1
     1
______
 48932


In [5]:
print(model)

BayesianTCN(
  (network): ModuleList(
    (0): BayesianTemporalBlock(
      (conv1): Conv1dReparameterization()
      (chomp1): Chomp1d()
      (relu1): ReLU()
      (dropout1): Dropout(p=0.5, inplace=False)
      (conv2): Conv1dReparameterization()
      (chomp2): Chomp1d()
      (relu2): ReLU()
      (dropout2): Dropout(p=0.5, inplace=False)
      (downsample): Conv1d(14, 32, kernel_size=(1,), stride=(1,))
      (final_relu): ReLU()
    )
    (1): BayesianTemporalBlock(
      (conv1): Conv1dReparameterization()
      (chomp1): Chomp1d()
      (relu1): ReLU()
      (dropout1): Dropout(p=0.5, inplace=False)
      (conv2): Conv1dReparameterization()
      (chomp2): Chomp1d()
      (relu2): ReLU()
      (dropout2): Dropout(p=0.5, inplace=False)
      (downsample): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
      (final_relu): ReLU()
    )
  )
  (avgpool): AdaptiveAvgPool1d(output_size=1)
  (mu): LinearReparameterization()
  (sigma): LinearReparameterization()
)


In [6]:
# 训练模型
import time
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family='Microsoft YaHei')

def model_train(epochs, model, optimizer, loss_function, train_loader, device):
    model = model.to(device)

    # 最低MSE
    minimum_mse = 1000.
    # 最佳模型
    best_model = model

    train_mse = []     # 记录在训练集上每个epoch的 MSE 指标的变化情况   平均值
  
    # 计算模型运行时间
    start_time = time.time()
    for epoch in range(epochs):
         # 训练
        model.train()
        train_loss = []    #保存当前epoch的MSE loss和
        for seq, labels in train_loader: 
            seq, labels = seq.to(device), labels.to(device)
            # 每次更新参数前都梯度归零和初始化
            optimizer.zero_grad()
            # 前向传播
            mu, sigma, kl = model(seq)  #   torch.Size([16, 10])
            kl_loss = model.kl_loss()
            # 损失计算
            nll_loss = loss_function(labels, mu, sigma)
            loss = nll_loss + kl_weight * kl_loss
            train_loss.append(loss.item()) # 计算 MSE 损失
            # 反向传播和参数更新
            loss.backward()
            optimizer.step()
        #     break
        # break
        # 计算总损失
        train_av_mseloss = np.average(train_loss) # 平均
        train_mse.append(train_av_mseloss)
        print(f'Epoch: {epoch+1:2} train_MSE-Loss: {train_av_mseloss:10.8f}')
       
        # 如果当前模型的 MSE 低于于之前的最佳准确率，则更新最佳模型
        #保存当前最优模型参数
        if train_av_mseloss < minimum_mse:
            minimum_mse = train_av_mseloss
            best_model = model# 更新最佳模型的参数
         
    # 保存最后的参数
    # torch.save(model.state_dict(), 'final_model_tcn.pt')
    # 保存最好的参数
    torch.save(best_model.state_dict(), 'best_model_tcn.pt')
    print(f'\nDuration: {time.time() - start_time:.0f} seconds')
    
    # 可视化
    plt.plot(range(epochs), train_mse, color = 'b',label = 'train_MSE-loss')
    plt.legend()
    plt.show()   #显示 lable 
    print(f'min_MSE: {minimum_mse}')

#  模型训练
epochs = 500
model_train(epochs, model, optimizer, loss_function, train_loader, device)

Epoch:  1 train_MSE-Loss: 0.62332242
Epoch:  2 train_MSE-Loss: 0.40798503
Epoch:  3 train_MSE-Loss: 0.33776862
Epoch:  4 train_MSE-Loss: 0.19790356
Epoch:  5 train_MSE-Loss: 0.17269044
Epoch:  6 train_MSE-Loss: 0.15064684
Epoch:  7 train_MSE-Loss: 0.04216006
Epoch:  8 train_MSE-Loss: -0.07038242
Epoch:  9 train_MSE-Loss: -0.05683769
Epoch: 10 train_MSE-Loss: -0.12251263
Epoch: 11 train_MSE-Loss: -0.16178051
Epoch: 12 train_MSE-Loss: -0.22419164
Epoch: 13 train_MSE-Loss: -0.24110943
Epoch: 14 train_MSE-Loss: -0.11570708
Epoch: 15 train_MSE-Loss: -0.12913435
Epoch: 16 train_MSE-Loss: -0.29790810


KeyboardInterrupt: 

### 第一部分，训练集评估

In [64]:
# 模型预测
# 模型 测试集 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型的状态字典
model.load_state_dict(torch.load('best_model_tcn.pt'))
model = model.to(device)

# 预测数据
origin_data = []
pre_data = []
with torch.no_grad():
        for data, label in train_loader:
            # 原始标签
            origin_lable = label.tolist()
            origin_data += origin_lable
            model.eval()  # 将模型设置为评估模式
            
            # 预测
            data, label = data.to(device), label.to(device)
            mu, sigma, kl = model(data)  # 对测试集进行预测
            train_pred = mu.tolist()
            # train_pred = train_pred.tolist()
            pre_data += train_pred      

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# 反归一化处理
# 使用相同的均值和标准差对预测结果进行反归一化处理
# 反标准化
scaler  = load('../dataresult/scaler')
origin_data = scaler.inverse_transform(origin_data)
pre_data = scaler.inverse_transform(pre_data)

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 模型分数
score = r2_score(origin_data, pre_data)
print('训练集上 模型分数-R^2:',score)

print('*'*50)
# 训练集上的预测误差
train_mse = mean_squared_error(origin_data, pre_data)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(origin_data, pre_data)
print('训练数据集上的均方误差-MSE: ',train_mse)
print('训练数据集上的均方根误差-RMSE: ',train_rmse)
print('训练数据集上的平均绝对误差-MAE: ',train_mae)

# 测试集评估

In [66]:
# 模型 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型的状态字典
model.load_state_dict(torch.load('best_model_tcn.pt'))
model = model.to(device)

# 预测数据
test_origin_data = []
test__pre_data = []
with torch.no_grad():
        for data, label in test_loader:
            # 原始标签
            origin_lable = label.tolist()
            test_origin_data += origin_lable
            model.eval()  # 将模型设置为评估模式
            
            # 预测
            data, label = data.to(device), label.to(device)
            mu, sigma, kl = model(data)  # 对测试集进行预测
            pred = mu.tolist()
            test__pre_data += pred        

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# 反归一化处理
# 使用相同的均值和标准差对预测结果进行反归一化处理
# 反标准化
scaler  = load('../dataresult/scaler')
test_origin_data = scaler.inverse_transform(test_origin_data)
test__pre_data = scaler.inverse_transform(test__pre_data)

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 模型分数
score = r2_score(test_origin_data, test__pre_data)
print('测试集上 模型分数-R^2:',score)

print('*'*50)
# 训练集上的预测误差
test_mse = mean_squared_error(test_origin_data, test__pre_data)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(test_origin_data, test__pre_data)
print('测试数据集上的均方误差-MSE: ',test_mse)
print('测试数据集上的均方根误差-RMSE: ',test_rmse)
print('测试数据集上的平均绝对误差-MAE: ',test_mae)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family='Microsoft YaHei')

# 可视化结果
plt.figure(figsize=(12, 6), dpi=100)
plt.plot(test_origin_data, label='真实寿命',color='pink')  # 真实值
plt.plot(test__pre_data, label='TCN 预测值',color='c')  # 预测值

# plt.plot([-1,170],[2.0*0.7,2.0*0.7],c='black',lw=1,ls='--')  # 临界点直线  可自己调整位置

plt.xlabel('运行周期/10s', fontsize=12)
plt.ylabel('寿命百分比', fontsize=12)
plt.title('Bearing1_3 预测结果', fontsize=16)
plt.legend()
plt.show()

# 保存数据
# 保存数据
dump(test_origin_data, '../画图对比/tcn_origin') 
dump(test__pre_data, '../画图对比/tcn_pre') 